In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用CPU和Vertex AI上的多节点NCCL分布式数据并行训练的PyTorch图像分类

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/multi_node_ddp_nccl_vertex_training_with_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftraining%2Fmulti_node_ddp_nccl_vertex_training_with_custom_container.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/multi_node_ddp_nccl_vertex_training_with_custom_container.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/multi_node_ddp_nccl_vertex_training_with_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示如何使用Vertex AI SDK for Python和自定义容器创建一个分布式NCCL PyTorch训练作业。这有助于您的训练作业扩展以处理大量数据。

了解更多关于[Vertex AI训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用 Vertex AI SDK for Python 和自定义容器创建一个分布式 PyTorch 训练作业。您将设置一个自定义容器，一个 Vertex AI TensorBoard 实例，并运行一个自定义训练作业。

本教程使用以下 Vertex AI 服务：

- Vertex AI TensorBoard
- Vertex AI 训练

执行的步骤包括：

- 使用 Artifact Registry 和 Docker 构建自定义容器。
- 创建一个 Vertex AI TensorBoard 实例来存储您的 Vertex AI 实验。
- 使用 Vertex AI SDK for Python 运行一个 Vertex AI 训练作业。

数据集

本教程使用的数据集是<a href="http://yann.lecun.com/exdb/mnist/">MNIST数据库</a>。手写数字MNIST数据库有一个包含60,000个示例的训练集，和一个包含10,000个示例的测试集。该数据集是从手写数字的二进制图像构建的较大数据集的子集。这些数字已经被尺寸归一化，并居中于固定尺寸的图像中。

### 成本

本教程使用谷歌云的计费组件：

* Vertex AI

* 云存储

* Vertex AI TensorBoard（请注意，Vertex AI TensorBoard 每月收取唯一活跃用户每月300美元的费用。活跃用户通过Vertex AI TensorBoard UI 进行测量。您还需要为在云存储中存储的TensorBoard日志和与Vertex AI TensorBoard一起使用的谷歌云资源付费。 <a href='https://cloud.google.com/vertex-ai/pricing#tensorboard'>查看最新价格。</a>)

了解有关[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和[云存储价格](https://cloud.google.com/storage/pricing)的信息，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/),  根据您的预期使用情况生成一个成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
# Install the Vertex AI SDK for Python
! pip3 install --quiet --upgrade google-cloud-aiplatform

# Install other required libraries
! pip3 install --quiet torch \
                        torchvision \
                        tensorboard

### 重启运行时（仅适用于Colab）

为了使用新安装的软件包，您必须在Google Colab上重启运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续下一步之前，请等待完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google Cloud项目信息

了解更多有关[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

初始化 Vertex AI SDK for Python

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目，并启用 [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=LOCATION)

### 服务帐户

您可以使用服务帐户来运行Vetex AI CustomContainerTrainingJob。如果您不想使用您项目的Compute Engine服务帐户，将`SERVICE_ACCOUNT`设置为另一个服务帐户ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 启用Artifact Registry API
您必须为您的项目启用[Artifact Registry API](https://cloud.google.com/artifact-registry/docs/reference/rest)服务。

<a href="https://cloud.google.com/artifact-registry/docs/enable-service">了解更多关于启用服务的信息</a>。

In [ ]:
import os

! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

创建训练应用程序

在进行本地训练之前，请为您的训练应用程序创建一个源代码文件、一个需求文件和一个 Docker 文件。

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = "trainer"

# Create the app folder
!mkdir -p $PYTHON_PACKAGE_APPLICATION_DIR

### 创建训练脚本

创建运行PyTorch分布式训练的训练文件。

In [ ]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/task.py


"""
Main program for PyTorch distributed training.
Adapted from: https://github.com/narumiruna/pytorch-distributed-example
"""

import argparse
import os
import shutil

import torch
from torch import distributed
from torch.nn.parallel import DistributedDataParallel
from torch.utils import data
from torch.utils.tensorboard import SummaryWriter

from torchvision import datasets, transforms

def parse_args():

  parser = argparse.ArgumentParser()

  # Using environment variables for Cloud Storage directories
  # see more details in https://cloud.google.com/vertex-ai/docs/training/code-requirements
  parser.add_argument(
      '--model-dir', default=os.getenv('AIP_MODEL_DIR'), type=str,
      help='a Cloud Storage URI of a directory intended for saving model artifacts')
  parser.add_argument(
      '--tensorboard-log-dir', default=os.getenv('AIP_TENSORBOARD_LOG_DIR'), type=str,
      help='a Cloud Storage URI of a directory intended for saving TensorBoard')
  parser.add_argument(
      '--checkpoint-dir', default=os.getenv('AIP_CHECKPOINT_DIR'), type=str,
      help='a Cloud Storage URI of a directory intended for saving checkpoints')

  parser.add_argument(
      '--backend', type=str, default='gloo',
      help='Use the `nccl` backend for distributed GPU training.'
           'Use the `gloo` backend for distributed CPU training.')
  parser.add_argument(
      '--init-method', type=str, default='env://',
      help='URL specifying how to initialize the package.')
  parser.add_argument(
      '--world-size', type=int, default=os.environ.get('WORLD_SIZE', 1),
      help='The total number of nodes in the cluster. '
           'This variable has the same value on every node.')
  parser.add_argument(
      '--rank', type=int, default=os.environ.get('RANK', 0),
      help='A unique identifier for each node. '
           'On the master worker, this is set to 0. '
           'On each worker, it is set to a different value from 1 to WORLD_SIZE - 1.')
  parser.add_argument(
      '--epochs', type=int, default=20)
  parser.add_argument(
      '--no-cuda', action='store_true')
  parser.add_argument(
      '-lr', '--learning-rate', type=float, default=1e-3)
  parser.add_argument(
      '--batch-size', type=int, default=128)
  parser.add_argument(
      '--local-mode', action='store_true', help='use local mode when running on your local machine')

  args = parser.parse_args()

  return args

def makedirs(model_dir):
  if os.path.exists(model_dir) and os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
  os.makedirs(model_dir)
  return

def distributed_is_initialized():
  if distributed.is_available():
    if distributed.is_initialized():
      return True
  return False

class Average(object):

  def __init__(self):
    self.sum = 0
    self.count = 0

  def __str__(self):
    return '{:.6f}'.format(self.average)

  @property
  def average(self):
    return self.sum / self.count

  def update(self, value, number):
    self.sum += value * number
    self.count += number

class Accuracy(object):

  def __init__(self):
    self.correct = 0
    self.count = 0

  def __str__(self):
    return '{:.2f}%'.format(self.accuracy * 100)

  @property
  def accuracy(self):
    return self.correct / self.count

  @torch.no_grad()
  def update(self, output, target):
    pred = output.argmax(dim=1)
    correct = pred.eq(target).sum().item()

    self.correct += correct
    self.count += output.size(0)

class Net(torch.nn.Module):

  def __init__(self, device):
    super(Net, self).__init__()
    self.fc = torch.nn.Linear(784, 10).to(device)

  def forward(self, x):
    return self.fc(x.view(x.size(0), -1))

class MNISTDataLoader(data.DataLoader):

  def __init__(self, root, batch_size, train=True):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ])

    dataset = datasets.MNIST(root, train=train, transform=transform, download=True)
    sampler = None
    if train and distributed_is_initialized():
      sampler = data.DistributedSampler(dataset)

    super(MNISTDataLoader, self).__init__(
        dataset,
        batch_size=batch_size,
        shuffle=(sampler is None),
        sampler=sampler,
    )

class Trainer(object):

  def __init__(self,
      model,
      optimizer,
      train_loader,
      test_loader,
      device,
      model_name,
      checkpoint_path
  ):
    self.model = model
    self.optimizer = optimizer
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.device = device
    self.model_name = model_name
    self.checkpoint_path = checkpoint_path

  def save(self, model_dir):
    model_path = os.path.join(model_dir, self.model_name)
    torch.save(self.model.state_dict(), model_path)

  def fit(self, epochs, is_chief, writer):

    for epoch in range(1, epochs + 1):

      print('Epoch: {}, Training ...'.format(epoch))
      train_loss, train_acc = self.train()

      if is_chief:
        test_loss, test_acc = self.evaluate()
        writer.add_scalar('Loss/train', train_loss.average, epoch)
        writer.add_scalar('Loss/test', test_loss.average, epoch)
        writer.add_scalar('Accuracy/train', train_acc.accuracy, epoch)
        writer.add_scalar('Accuracy/test', test_acc.accuracy, epoch)
        torch.save(self.model.state_dict(), self.checkpoint_path)

        print(
            'Epoch: {}/{},'.format(epoch, epochs),
            'train loss: {}, train acc: {},'.format(train_loss, train_acc),
            'test loss: {}, test acc: {}.'.format(test_loss, test_acc),
        )

  def train(self):

    self.model.train()

    train_loss = Average()
    train_acc = Accuracy()

    for data, target in self.train_loader:
      data = data.to(self.device)
      target = target.to(self.device)

      output = self.model(data)
      loss = torch.nn.functional.cross_entropy(output, target)

      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()

      train_loss.update(loss.item(), data.size(0))
      train_acc.update(output, target)

    return train_loss, train_acc

  @torch.no_grad()
  def evaluate(self):
    self.model.eval()

    test_loss = Average()
    test_acc = Accuracy()

    for data, target in self.test_loader:
      data = data.to(self.device)
      target = target.to(self.device)

      output = self.model(data)
      loss = torch.nn.functional.cross_entropy(output, target)

      test_loss.update(loss.item(), data.size(0))
      test_acc.update(output, target)

    return test_loss, test_acc

def main():

  args = parse_args()

  local_data_dir = './tmp/data'
  local_model_dir = './tmp/model'
  local_tensorboard_log_dir = './tmp/logs'
  local_checkpoint_dir = './tmp/checkpoints'

  model_dir = args.model_dir or local_model_dir
  tensorboard_log_dir = args.tensorboard_log_dir or local_tensorboard_log_dir
  checkpoint_dir = args.checkpoint_dir or local_checkpoint_dir

  gs_prefix = 'gs://'
  gcsfuse_prefix = '/gcs/'
  if model_dir and model_dir.startswith(gs_prefix):
    model_dir = model_dir.replace(gs_prefix, gcsfuse_prefix)
  if tensorboard_log_dir and tensorboard_log_dir.startswith(gs_prefix):
    tensorboard_log_dir = tensorboard_log_dir.replace(gs_prefix, gcsfuse_prefix)
  if checkpoint_dir and checkpoint_dir.startswith(gs_prefix):
    checkpoint_dir = checkpoint_dir.replace(gs_prefix, gcsfuse_prefix)

  writer = SummaryWriter(tensorboard_log_dir)

  is_chief = args.rank == 0
  if is_chief:
    makedirs(checkpoint_dir)
    print(f'Checkpoints will be saved to {checkpoint_dir}')

  checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pt')
  print(f'checkpoint_path is {checkpoint_path}')

  if args.world_size > 1:
    print('Initializing distributed backend with {} nodes'.format(args.world_size))
    distributed.init_process_group(
          backend=args.backend,
          init_method=args.init_method,
          world_size=args.world_size,
          rank=args.rank,
      )
    print(f'[{os.getpid()}]: '
          f'world_size = {distributed.get_world_size()}, '
          f'rank = {distributed.get_rank()}, '
          f'backend={distributed.get_backend()} \n', end='')

  if torch.cuda.is_available() and not args.no_cuda:
    device = torch.device('cuda:{}'.format(args.rank))
  else:
    device = torch.device('cpu')

  model = Net(device=device)
  if distributed_is_initialized():
    model.to(device)
    model = DistributedDataParallel(model)

  if is_chief:
    # All processes should see same parameters as they all start from same
    # random parameters and gradients are synchronized in backward passes.
    # Therefore, saving it in one process is sufficient.
    torch.save(model.state_dict(), checkpoint_path)
    print(f'Initial chief checkpoint is saved to {checkpoint_path}')

  # Use a barrier() to make sure that process 1 loads the model after process
  # 0 saves it.
  if distributed_is_initialized():
    distributed.barrier()
    # configure map_location properly
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f'Initial chief checkpoint is saved to {checkpoint_path} with map_location {device}')
  else:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f'Initial chief checkpoint is loaded from {checkpoint_path}')

  optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

  train_loader = MNISTDataLoader(
      local_data_dir, args.batch_size, train=True)
  test_loader = MNISTDataLoader(
      local_data_dir, args.batch_size, train=False)

  trainer = Trainer(
      model=model,
      optimizer=optimizer,
      train_loader=train_loader,
      test_loader=test_loader,
      device=device,
      model_name='mnist.pt',
      checkpoint_path=checkpoint_path,
  )
  trainer.fit(args.epochs, is_chief, writer)

  if model_dir == local_model_dir:
    makedirs(model_dir)
    trainer.save(model_dir)
    print(f'Model is saved to {model_dir}')

  print(f'Tensorboard logs are saved to: {tensorboard_log_dir}')

  writer.close()

  if is_chief:
    os.remove(checkpoint_path)

  if distributed_is_initialized():
    distributed.destroy_process_group()

  return

if __name__ == '__main__':
  main()


创建需求文件

In [ ]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/requirements.txt

torch
torchvision
tensorboard


### 创建Dockerfile

In [ ]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/Dockerfile


FROM pytorch/pytorch:1.8.1-cuda11.1-cudnn8-runtime

RUN apt-get update && \
    apt-get install -y curl gnupg && \
    echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && \
    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key --keyring /usr/share/keyrings/cloud.google.gpg  add - && \
    apt-get update -y && \
    apt-get install google-cloud-sdk -y

COPY . /trainer

WORKDIR /trainer

RUN pip install -r requirements.txt

ENTRYPOINT ["python", "-m", "task"]


在本地运行您的训练脚本以测试您的应用程序。

In [ ]:
%run trainer/task.py --epochs 5 --no-cuda --local-mode 

In [ ]:
# List the outputs
! ls ./tmp

创建一个自定义图像用于训练

### 创建一个私有的Docker仓库

您的第一步是在Artifact Registry中创建自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令，使用您的区域创建一个新的Docker仓库，描述为"docker repository"。

2. 运行`gcloud artifacts repositories list`命令，验证您的仓库已经创建成功。

In [ ]:
# Set the name for your private repo
PRIVATE_REPO = "my-docker-repo-unique"

# Create the repository
! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={LOCATION} --description="Docker repository"

# List the repositories and verify
! gcloud artifacts repositories list

In [ ]:
DEPLOY_IMAGE = (
    f"{LOCATION}-docker.pkg.dev/" + PROJECT_ID + f"/{PRIVATE_REPO}" + "/tf_serving"
)

In [ ]:
print("Deployment:", DEPLOY_IMAGE)

在笔记本环境中执行

如果您在笔记本环境中运行本教程，例如在Vertex AI工作台中，请按照以下步骤进行。

如果您在Colab上运行，请跳转到“在Colab中执行”部分。

### 配置私有仓库的身份验证
在推送或拉取容器镜像之前，配置Docker使用gcloud命令行工具对您区域的Artifact Registry发出的请求进行身份验证。

In [ ]:
if not IS_COLAB:
    ! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet

### 构建和推送用于提供服务的镜像
设置用于提供预测的 TensorFlow Serving 镜像。

以下代码：
1. 从 Docker Hub 拉取相应的 CPU 或 GPU Docker 镜像用于 TF Serving。
2. 为将镜像与 Artifact Registry 注册创建标签。
3. 将镜像注册到 Artifact Registry。

<a href="https://www.tensorflow.org/tfx/serving/docker">了解有关 TensorFlow Serving 的更多信息</a>。

In [ ]:
if not IS_COLAB:
    ! cd trainer && docker build -t $DEPLOY_IMAGE -f Dockerfile .
    ! docker run --rm $DEPLOY_IMAGE --epochs 5 --no-cuda --local-mode
    ! docker push $DEPLOY_IMAGE

在Colab中执行

使用Cloud Build构建并推送Docker镜像。

In [ ]:
if IS_COLAB:
    ! cd trainer && gcloud builds submit --timeout=1800s --region={LOCATION} --tag $DEPLOY_IMAGE

创建一个 Vertex AI Tensorboard 实例

在下面为您的 Tensorboard 实例设置一个显示名称，然后运行 create 命令。

**注意:** 要使用现有的 Vertex AI Tensorboard 实例，请使用以下方法。

```
tensorboard_name = "您的 Tensorboard 资源名称或 Tensorboard 实例 ID"
tensorboard = aiplatform.Tensorboard(tensorboard_name=tensorboard_name)
```

In [ ]:
# Set display name for Tensorboard instance
TENSORBOARD_DISPLAY_NAME = "pytorch-image-classify-multi-node-ddp"

In [ ]:
# Create the Tensorboard instance
tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_DISPLAY_NAME,
)

创建一个顶点 AI 自定义容器训练作业

了解有关[为自定义训练配置计算资源的更多信息](https://cloud.google.com/vertex-ai/docs/training/configure-compute)。

In [ ]:
# Set display name for your training job
JOB_DISPLAY_NAME = "pytorch-image-classify-multi-node-ddp-training"
# Set a Cloud Storage output path
GCS_OUTPUT_URI_PREFIX = f"{BUCKET_URI}/{JOB_DISPLAY_NAME}"
# Set the number of replicas in the workerpool
REPLICA_COUNT = 1
# Set the machine type for running the training job
MACHINE_TYPE = "n1-standard-4"
# Set the number of accelerators needed for training
ACCELERATOR_COUNT = 1
# Set the accelerator type
ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
# Set the args to be passed to the training application
ARGS = [
    "--backend",
    "nccl",
    "--no-cuda",
    "--batch-size",
    "128",
    "--epochs",
    "25",
]

In [ ]:
# Create the custom container training job
custom_container_training_job = aiplatform.CustomContainerTrainingJob(
    display_name=JOB_DISPLAY_NAME,
    container_uri=DEPLOY_IMAGE,
)

运行训练任务

In [ ]:
custom_container_training_job.run(
    args=ARGS,
    base_output_dir=GCS_OUTPUT_URI_PREFIX,
    replica_count=REPLICA_COUNT,
    machine_type=MACHINE_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    accelerator_type=ACCELERATOR_TYPE,
    tensorboard=tensorboard.resource_name,
    service_account=SERVICE_ACCOUNT,
)

In [ ]:
print(f"Custom Training Job Name: {custom_container_training_job.resource_name}")
print(f"GCS Output URI Prefix: {GCS_OUTPUT_URI_PREFIX}")

查看培训输出结果物品

In [ ]:
! gsutil ls $GCS_OUTPUT_URI_PREFIX

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Vertex AI训练作业
- Vertex AI Tensorboard
- Cloud Storage存储桶
- Artifact Registry存储库

In [ ]:
# Delete the training job
custom_container_training_job.delete()

# Delete the Tensorboard instance
delete_tensorboard = False  # Set True for deletion

if delete_tensorboard:
    tensorboard.delete()

# Delete the Cloud Storage bucket
delete_bucket = False  # Set True for deletion

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Delete the repository in Artifact Registry
! gcloud artifacts repositories delete {PRIVATE_REPO} --location {LOCATION} --quiet

# Remove the outputs from the local execution
! rm -rf ./tmp

# Remove the training app folder
! rm -rf ./trainer